## **Initializarion**

In [1]:
from PIL import Image, ImageFilter
import numpy as np

## **Download an Random Image From Web.**

In [2]:
!wget https://file-examples-com.github.io/uploads/2017/10/file_example_TIFF_1MB.tiff

--2021-12-03 08:23:05--  https://file-examples-com.github.io/uploads/2017/10/file_example_TIFF_1MB.tiff
Resolving file-examples-com.github.io (file-examples-com.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to file-examples-com.github.io (file-examples-com.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1131930 (1.1M) [image/tiff]
Saving to: ‘file_example_TIFF_1MB.tiff’

file_example_TIFF_1 100%[===================>]   1.08M  --.-KB/s    in 0.03s   

2021-12-03 08:23:05 (32.3 MB/s) - ‘file_example_TIFF_1MB.tiff’ saved [1131930/1131930]



In [3]:
!mv file_example_TIFF_1MB.tiff blind.tiff

## **Declaring Required Functions.**

In [4]:
def summer(mat, d):
  irp, icp = mat.shape
  temp = np.zeros(mat.shape)

  for i in range(icp):
    temp[0, i] = np.sum(mat[:d, i])
    for j in range(1, irp-d+1):
        temp[j, i] = temp[j-1, i] + mat[d+j-1, i] - mat[j-1, i]
  return temp

def custom_var(img, d, tau):
  pow_1 = np.pad(img, d // 2, mode='constant').astype('int32')
  pow_2 = pow_1 ** 2

  ir, ic = img.shape
  irp, icp = pow_1.shape
  
  pow_1_sum = summer(pow_1, d)
  pow_2_sum = summer(pow_2, d)

  res_var = np.zeros(img.shape)

  for i in range(ir):
    var_1 = np.sum(pow_1[i:i+d, :d])
    var_2 = np.sum(pow_2[i:i+d, :d])
    res_var[i, 0] = abs(var_2 / 49 - (var_1**2) / 2401)
    for j in range(1, ic):
        var_1 = var_1 - pow_1_sum[i, j-1] + pow_1_sum[i, j+d-1]
        var_2 = var_2 - pow_2_sum[i, j-1] + pow_2_sum[i, j+d-1]
        res_var[i, j] = abs(var_2 / 49 - (var_1**2) / 2401)
  return (res_var >= tau).astype('int8')


In [5]:
def detector(img, d, tau):
  ir, ic = img.shape

  # Calculate nabla matrices
  temp_row = np.concatenate((np.zeros((ir, 1), dtype='int16'), img[:, :-1]), axis=1)
  temp_col = np.concatenate((np.zeros((1, ic), dtype='int16'), img[:-1, :]), axis=0)
  nabla_row = (img - temp_row == 0).astype('int8')
  nabla_col = (img - temp_col == 0).astype('int8')

  # Calculate Variance matrix
  var = custom_var(img, d, tau)

  # Calculate f value
  fr = np.sum(nabla_row * var) / np.sum(var)
  fc = np.sum(nabla_col * var) / np.sum(var)

  return fr/(2 ** 0.5) + fc/(2 ** 0.5)

## **Extractig f Value For The Image "blind.tiff"**

In [6]:
d, tau = 7, 100

img = Image.open('blind.tiff').convert('L')
f = detector(np.array(img), d, tau)

print('The f value for an unknown image "original.png" is :', f)

The f value for an unknown image "original.png" is : 0.25108240994428205


So final f value is > 0.2, which means the image is tampered with median filter.